In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
from collections import Counter
import numpy as np
import numpy.linalg as LA
import matplotlib
import matplotlib.pyplot as plt
#import geoplot as gplt
import glob
import datetime as dt
import random
from sklearn.cluster import KMeans
import math

In [ ]:
class flussi():
    
    impianti_file = 'impianti.csv'
    impianti_df = pd.read_csv(impianti_file).drop('Note', axis = 1)
    geo_start = gpd.points_from_xy(impianti_df['Latitude Start'], impianti_df['Longitude Start'])
    geo_end = gpd.points_from_xy(impianti_df['Latitude Start'], impianti_df['Longitude Start'])
    
    impianti_df = gpd.GeoDataFrame(impianti_df)

    impianti_df['GEO_START'] = geo_start
    impianti_df['GEO_END'] = geo_end
    
    impianto_line = gpd.GeoSeries(impianti_df.apply(lambda row: LineString([row['GEO_START'], row['GEO_END']]), axis=1))
    impianti_df['IMPIANTO_LINE'] = impianto_line

    
    def __init__(self, input_file, sample = None):
        '''Carica il file dei dati ed estrae le quantita di interesse'''
        self.file_name = input_file.split('/')[-1][0:-4]
        self.impianti = self.file_name[11:]
        self.date = self.file_name[0:10]
        
        self.df = pd.read_csv(input_file)
        self.df['DATA_ORA'] = pd.to_datetime(self.df['DATA_ORA'], format = "%d/%m/%Y %H.%M.%S")
            
        self.N = len(self.df)
        
        self.df = pd.merge(self.df, flussi.impianti_df, left_on = 'ID_IMPIANTO', right_on = 'Codice')
        
        geo_start = gpd.points_from_xy(self.df['Latitude Start'], self.df['Longitude Start'])
        geo_end = gpd.points_from_xy(self.df['Latitude End'], self.df['Longitude End'])
        self.df = gpd.GeoDataFrame(self.df)
        self.df['GEO START'] = geo_start
        self.df['GEO END'] = geo_end
        
        if VERBOSE>2:
            print(f'\n Caricato: {self.file_name}. Esempio record prima risalita:')
            print(self.df.loc[0])
        
        if sample:
            self.serie_set = list(set(self.df['SERIE']))
            self.df=self.df[self.df['SERIE'].isin(random.sample(self.serie_set, sample))]
        else:
            self.serie_set = list(set(self.df['SERIE']))
        
        self.cassa = self.df['CASSA']
        self.cassa_set = list(set(self.cassa))
        self.N_casse = len(self.cassa_set)
        self.cassa_counter = Counter(self.cassa)
        
        # la serie e' la ID dello sciatore
        self.serie = self.df['SERIE']
        #self.serie_set = list(set(self.serie))
        self.N_sciatori = len(self.serie_set)
        self.serie_counter = Counter(self.serie)

        self.data_ora = self.df['DATA_ORA']
        self.data_ora_set = list(set(self.data_ora))

        self.pool = self.df['POOL']
        self.pool_set = list(set(self.pool))

        self.impianto = self.df['ID_IMPIANTO']
        self.impianto_set = list(set(self.impianto))
        self.N_impianti = len(self.impianto_set)
        self.impianto_counter = Counter(self.impianto)

        self.lettore = self.df['ID_LETTORE']
        self.lettore_set = list(set(self.lettore))
        
        self.flow_matrix, self.flow_matrices, self.skier_num_runs = self.get_flow_matrix()
        
        if VERBOSE>1:
            print(f'Sommario {self.file_name} -- {self.N} risalite per {self.N_sciatori} sciatori su {self.N_impianti} impianti di risalita.')
        
    def show_stats(self):
        '''Show some basic statistics of the data'''
        
        ## Contare le risalite per sciatore
        corse = np.zeros((self.N_sciatori))
        for i in range(self.N_sciatori):
            corse[i] = self.serie_counter[self.serie_set[i]]
    
        corse = np.flip(np.sort(corse))
        
        
        plt.subplot(1,2,1)
        plt.plot(range(1,self.N_sciatori+1), corse,lw=3,label=self.date)
        plt.xscale('log')
        
        plt.title(f'Risalite per sciatore - {self.impianti}',fontsize=24)
        plt.xlabel('Indice sciatore',fontsize=20)
        plt.ylabel('Numero risalite',fontsize=20)
        
        ## Contare le risalite per impianto
        corse = np.zeros((self.N_impianti))
        for i in range(self.N_impianti):
            corse[i] = self.impianto_counter[self.impianto_set[i]]
    
        corse = np.flip(np.sort(corse))
        
        # plotta le risalite per impianto
        plt.subplot(1,2,2)
        plt.plot(corse,lw=3,label=self.date)
        
        plt.title(f'Risalite per impianto - {self.impianti}',fontsize=24)
        plt.xlabel('Indice impianto',fontsize=20)
        plt.ylabel('Numero risalite',fontsize=20)
        
    def create_sample(self, size):
        return flussi('./file csv/'+self.file_name+'.txt', size)
    
    def build_flow_init(self):
        df = self.df
        zer = np.zeros((len(self.impianto_set),len(self.impianto_set)))
        flow_matrix = pd.DataFrame(data = zer, columns = self.impianto_set)
        flow_matrix['ind'] = self.impianto_set
        flow_matrix.set_index('ind', inplace = True, drop = True)
        return flow_matrix

    def get_flow_matrix(self, num_skiers = None):
        if num_skiers:
            serie_sample = random.sample(self.serie_set, num_skiers)
        else:
            serie_sample = self.serie_set
        
        df = self.df
        flow_matrix = self.build_flow_init()
        flow_matrices={}
        skier_num_runs={}
        
        # loop over each skier
        for s in serie_sample:

            # select the sequence of rides for skier s sorted by time
            ski_day = df[df['SERIE'] == s].sort_values('DATA_ORA')
            skier_num_runs[s] = len(ski_day)
            current_fm = self.build_flow_init()
            flow_matrices[s] = current_fm

            for i in range(len(ski_day) - 1):
                x = ski_day['ID_IMPIANTO'].iloc[i]
                y = ski_day['ID_IMPIANTO'].iloc[i+1]
                flow_matrix[y][x]+=1
                current_fm[y][x]+=1

        return flow_matrix, flow_matrices, skier_num_runs
    

In [ ]:
INPUT_DIR = './file csv/'
FILE_STRINGS = ['2006-SelvaColfosco','2006-Selva']
VERBOSE = 0

k=0
dataset_list = []
for str in FILE_STRINGS: ## each str is a group of files

    INPUT_FILES = glob.glob(INPUT_DIR + f'*{str}.txt') # group of files chosen by str

    fig = plt.figure(figsize=(20,10))
   
    for f in INPUT_FILES[4:5]:
        fl = flussi(f)
        fl.show_stats()
        dataset_list.append(fl)
        print(k)
        k+=1
    
    for i in range(1,3):
        plt.subplot(1,2,i)
        plt.legend(fontsize=18)

In [ ]:
dataset_list[0].df

In [ ]:
flussi.impianti_df

In [ ]:
def build_flow_init(flusso):
    df = flusso.df
    zer = np.zeros((len(flusso.impianto_set),len(flusso.impianto_set)))
    flow_matrix = pd.DataFrame(data = zer, columns = flusso.impianto_set)
    flow_matrix['ind'] = flusso.impianto_set
    flow_matrix.set_index('ind', inplace = True, drop = True)
    return flow_matrix

def get_flow_matrix(flusso, num_skiers = None):
    if num_skiers:
        serie_sample = random.sample(flusso.serie_set, num_skiers)
    else:
        serie_sample = flusso.serie_set
    
    df = flusso.df
    flow_matrix = build_flow_init(flusso)
    flow_matrices={}
    skier_num_runs={}
    
    print(flusso.N)
    
    # loop over each skier
    for s in serie_sample:
        
        # select the sequence of rides for skier s sorted by time
        ski_day = df[df['SERIE'] == s].sort_values('DATA_ORA')
        skier_num_runs[s] = len(ski_day)
        current_fm = build_flow_init(dataset_list[0])
        flow_matrices[s] = current_fm
    
        for i in range(len(ski_day) - 1):
            x = ski_day['ID_IMPIANTO'].iloc[i]
            y = ski_day['ID_IMPIANTO'].iloc[i+1]
            flow_matrix[y][x]+=1
            current_fm[y][x]+=1
            
    return flow_matrix, flow_matrices, skier_num_runs

In [ ]:
def get_skier_diff(flusso, num_skiers = None):
    #flow_matrix = flusso.flow_matrix
    flow_matrices = flusso.flow_matrices
    skier_num_runs = flusso.skier_num_runs

    if num_skiers:
        serie_sample = random.sample(list(flusso.skier_num_runs.keys()), num_skiers)
    else:
        serie_sample = list(flusso.skier_num_runs.keys())
    
    #df = flusso.df
    
    zer = np.zeros((len(serie_sample),len(serie_sample)))
    diff_matrix = pd.DataFrame(data = zer, columns = serie_sample)
    diff_matrix['ind'] = list(serie_sample)
    diff_matrix.set_index('ind', inplace = True, drop = True)
    
    k=1
    for i in serie_sample:
        for j in serie_sample[k:]:
            #dist = LA.norm(flow_matrices[i] - flow_matrices[j])
            #diff_runs = np.abs(skier_num_runs[i] - skier_num_runs[j])
            #num_runs = skier_num_runs[i] + skier_num_runs[j]
            #diff_matrix[j][i] = (dist + (1/num_runs)) / (num_runs/2)
            i_log = np.log2(flow_matrices[i].to_numpy()+1)
            j_log = np.log2(flow_matrices[j].to_numpy()+1)
            diff_matrix[j][i] = LA.norm(i_log - j_log)
            diff_matrix[i][j] = LA.norm(i_log - j_log)
        #print(k,'/',len(serie_sample))
        k+=1
    return diff_matrix

In [ ]:
dist_df = get_skier_diff(dataset_list[0], 100)
dist_df

In [ ]:
dist_mat = dist_df.to_numpy()
dist_arr = np.array([])
for i in range(len(dist_mat)):
    for j in range(i+1,len(dist_mat)):
        dist_arr = np.append(dist_arr, dist_mat[i][j])
plt.hist(dist_arr, bins = 25)
plt.show

In [ ]:
flusso = dataset_list[0]
X = np.empty((0,flusso.N_impianti**2), int)
for serie in flusso.serie_set:
    fm_unrolled = flusso.flow_matrices[serie].to_numpy().ravel()
    X = np.append(X, [fm_unrolled], axis = 0)

In [ ]:
U,s,VT=LA.svd(X)

In [ ]:
kmeans = KMeans(n_clusters=8, random_state=0).fit(X)
centroids = [cent.reshape((flusso.N_impianti, flusso.N_impianti)) for cent in kmeans.cluster_centers_]

In [ ]:
def centroids_as_flow_mat(cent_list):
    flow_mat_list = []
    for c in cent_list:
        c_trunc = np.trunc(c*10)/10
        c_flow_matrix = pd.DataFrame(data = c_trunc, columns = flusso.impianto_set)
        c_flow_matrix['ind'] = flusso.impianto_set #flusso needs to be parameterized
        c_flow_matrix.set_index('ind', inplace = True, drop = True)
        flow_mat_list.append(c_flow_matrix)
    return flow_mat_list

centroids_as_flow_mat(centroids)

In [ ]:
'''x = [np.dot(x_i, VT.T[0]) for x_i in X] #check if these are actually the PCs
y = [np.dot(x_i, VT.T[1]) for x_i in X]
plt.scatter(x = x, y = y)
for c in centroids:
    plt.scatter( np.dot(c.ravel(), VT.T[0]), np.dot(c.ravel(), VT.T[1]), c = 'r')

plt.show()'''

In [ ]:
def plot_impianti(ax = None):
    for row in flussi.impianti_df.iterrows():
        lat_start  = row[1]['Latitude Start']
        long_start = row[1]['Longitude Start']
        lat_end    = row[1]['Latitude End']
        long_end   = row[1]['Longitude End']
        
        mid_lat = np.mean([lat_start, lat_end])
        mid_long = np.mean([long_start, long_end])
        
        geometry = LineString([(long_start, lat_start),(long_end, lat_end)])
        gdf = gpd.GeoDataFrame(geometry=[geometry])
        ax = gdf.plot(ax = ax, lw = 5, figsize = (12,12))
        ax.text(s=row[1]['Nome'],x = mid_long, y = mid_lat)
    return ax

def plot_flow_matrix(flow_matrix, ax):
    max_flow = flow_matrix.max().max()
    print(max_flow)
    
    ax = plot_impianti(ax)
    idf = flussi.impianti_df
    
    for i in flow_matrix:
        for j in flow_matrix:
            row_i = idf[idf['Codice'] == i]
            row_j = idf[idf['Codice'] == j]
            
            #print(int(row_j['Latitude Start']))
            
            lat_start  = float(row_i['Latitude Start'])
            long_start = float(row_i['Longitude Start'])
            lat_end    = float(row_j['Latitude End'])
            long_end   = float(row_j['Longitude End'])
            
            #print(lat_start, long_start, lat_end, long_end  )

            mid_lat = np.mean([lat_start, lat_end])
            mid_long = np.mean([long_start, long_end])
            
            geometry = LineString([(long_start, lat_start),(long_end, lat_end)])
            gdf = gpd.GeoDataFrame(geometry=[geometry])
            
            
            ax = gdf.plot(ax = ax, lw = 5*flow_matrix[i][j]/max_flow, figsize = (12,12), color='r', alpha = 0.5)
            
            #plt.scatter(x=11.8, y=46.55)
            
def plot_centroids(centroids):
    f = plt.figure(figsize=(20,20))
    f.title
    len_cent = len(centroids)
    for i, cent in enumerate(centroids):
        ax = f.add_subplot(len_cent, 1, i+1)
        plot_flow_matrix(cent, ax)
    plt.show()
    
plot_centroids(centroids_as_flow_mat(centroids))

In [ ]:
plot_impianti()

In [ ]:
flussi.impianti_df

In [ ]:
flow_matrix = centroids_as_flow_mat(centroids)[1]
for i in flow_matrix:
    for j in flow_matrix.keys():
        print(flow_matrix[i][j])

In [ ]:
flussi.impianti_df

In [ ]:
def last_run_before_time(imp_entrances, time):
    '''
    Finds last entrance in IMP_ENTRANCES before TIME.
    
    parameters:
    IMP_ENTRANCES: a dataframe of a specific skiers entrances onto a specific lift
             TIME: the threshold time
             
    returns:
        I: The index of the last entrance in IMP_ENTRANCES before TIME
    DELTA: The time difference between the desired entrance and TIME
    '''
    delta = dt.timedelta(0)
    t = imp_entrances['DATA_ORA'].iloc[0]
    i = 0
    while t < time and i < len(imp_entrances) - 1:
        i+=1
        delta = time - t
        t = imp_entrances['DATA_ORA'].iloc[i]
    return i, delta

def sellar_anti_class(day, inter_ids):
    '''
    Tries to find skiers who are on the anticlockwise sellaronda on day with index DAY, by examining time
    delta between skiers entrances onto Borest and Ciampinoi. Returns number of rides on lifts given by
    INTER_IDS between Borest and Ciampinoi.
    
    parameters:
          DAY: The index of the day to be analyzed in DATASET_LIST
    INTER_IDS: ID_IMPIANTO of the lifts whose intermediate rides should be counted
    
    returns:
    
    '''
    flusso = dataset_list[day]
    df = flusso.df

    delta_arr = np.array([])
    num_danter_arr = np.array([])
    
    inter_counts = {}
    for inter_id in inter_ids:
        inter_counts[inter_id] = np.array([])

    k=0
    for s in flusso.serie_set:
        if k % 1000 == 0:
            print(k, '/', len(flusso.serie_set))
        k+=1

        ski_day = df[df['SERIE'] == s]
        borest = ski_day[ski_day['ID_IMPIANTO'] == 175]
        ciampi = ski_day[ski_day['ID_IMPIANTO'] == 220]
        
        danter = ski_day[ski_day['ID_IMPIANTO'] == 221]
        cir = ski_day[ski_day['ID_IMPIANTO'] == 232]

        if len(borest) == 0 or len(ciampi) == 0:
            continue

        i, delta = last_run_before_time(borest, ciampi['DATA_ORA'].iloc[0])
        last_borest_time = borest['DATA_ORA'].iloc[i]

        if delta == dt.timedelta(0):
            continue

        def num_inter_runs(inter_id):
            inter_all = ski_day[ski_day['ID_IMPIANTO'] == inter]
            inter_lower = inter_all[inter_all['DATA_ORA'] > last_borest_time]
            inter = inter_lower[inter_lower['DATA_ORA'] < ciampi['DATA_ORA'].iloc[0]]
            return inter
        
        for inter_id in inter_ids:
            ic_arr = inter_counts[inter_id]
            ic_arr = np.append(ic_arr, num_inter_runs(inter_id))

        delta = delta.total_seconds()//60

        delta_arr = np.append(delta_arr, delta)
        
        return delta_arr, 

In [ ]:
plt.hist(num_danter_arr[num_danter_arr>0])
plt.show()

In [ ]:
flusso = dataset_list[6]
df = flusso.df
df['DATA_ORA'][0] - df['DATA_ORA'][120449] > dt.timedelta(0)

In [ ]:
def create_date_range(date_times, is_set = False):
    '''
    parameters:
    DATE_TIMES: Series or set of datetimes
    IS_SET: set to True if DATE_TIMES is a set
    
    return value: a list of datetimes ranging from 
    the earliest to the latest time in DATE_TIMES
    with frequency of one minute
    '''
    if not is_set:
        dt_set = set(date_times)
    else:
        dt_set = date_times
    return pd.date_range(min(dt_set), max(dt_set), freq = 'T')

def skier_path_matrix(flusso, num_skiers = None):
    #TEMP:
    #day_frame = day_frame[day_frame['SERIE'] == 33022]
    day_frame = flusso.df
    if num_skiers:
        day_frame = flusso.df[flusso.df['SERIE'] == random.sample(flusso.serie_set, num_skiers)]
    
    serie = day_frame['SERIE']
    serie_set = list(set(serie))
    data_ora = day_frame['DATA_ORA']
    data_ora_set = set(data_ora)
    impianti = day_frame['ID_IMPIANTO']
    
    dr = create_date_range(data_ora_set, is_set = True)
    
    matrix = pd.DataFrame(columns = dr)
    matrix_norm = pd.DataFrame(columns = np.arange(len(dr)))
    
    k = 0
    max_day = 0
    for skier in serie_set:
        k+=1
        if k%1000 == 0:
            print(f'{k}/{len(serie_set)}')
        ski_day = day_frame[serie == skier].sort_values('DATA_ORA')
        ski_day_ore = ski_day['DATA_ORA']
        
        #print(ski_day)
        
        td_init = int((ski_day_ore.iloc[0] - dr[0]).total_seconds()/60)
        impianti_skier = np.array(np.zeros(td_init))
            
        i=-1
        day_len = 0
        for i in range(len(ski_day) - 1):
            td = int((ski_day_ore.iloc[i+1] - ski_day_ore.iloc[i]).total_seconds()/60)
            to_add = np.full(shape = td, fill_value = ski_day['ID_IMPIANTO'].iloc[i])
            impianti_skier = np.append(impianti_skier, to_add)
            
            day_len += td
        
        if day_len > max_day:
            max_day = day_len
            max_skier = skier
        
        #print(f'len diff: {len(dr) - len(impianti_skier)}, i: {i}, skier: {skier}')
        
        impianti_skier_norm = impianti_skier[td_init:]
        
        fv = ski_day['ID_IMPIANTO'].iloc[i+1]
        fill_norm = np.full(shape = len(dr) - len(impianti_skier_norm), fill_value = fv)
        fill = fill_norm[td_init:]
        
        impianti_skier_norm = np.append(impianti_skier_norm, fill_norm)
        #print(len(impianti_skier_norm), impianti_skier_norm)
        impianti_skier = np.append(impianti_skier, fill)

        matrix.loc[skier] = impianti_skier
        matrix_norm.loc[skier] = impianti_skier_norm
        
    return matrix, matrix_norm

In [ ]:
def skier_path_matrix_latlong(day_frame, num_skiers):
    day_frame = flusso.df
    if num_skiers:
        day_frame = flusso.df[flusso.df['SERIE'].isin(random.sample(flusso.serie_set, num_skiers))]
    
    serie = day_frame['SERIE']
    serie_set = list(set(serie))
    data_ora = day_frame['DATA_ORA']
    data_ora_set = set(data_ora)
    impianti = day_frame['ID_IMPIANTO']
    
    dr = create_date_range(data_ora_set, is_set = True)
    
    matrix = pd.DataFrame(columns = [dr,dr])
    #matrix_norm = pd.DataFrame(columns = np.arange(len(dr)))
    
    k = 0
    max_day = 0
    for skier in serie_set:
        print(skier)
    return matrix
        

In [ ]:
skier_path_matrix_latlong(dataset_list[6].df, 100)

In [ ]:
x = skier_path_matrix(dataset_list[6].df, 100)[0]

In [ ]:
spm = x
a,b=0,0
for i in spm.iterrows():
    for j in spm.iterrows():
        dist = 0
        for k in range(len(i[1])):
            i_pos = get_pos(i[1].iloc[k])
            j_pos = get_pos(j[1].iloc[k])
            dist += get_dist(i_pos, j_pos)
                

In [ ]:
flusso = dataset_list[6]

dat = flusso.df[flusso.df['SERIE'].isin(random.sample(flusso.serie_set, 5))]
dat.sort_values('SERIE')

In [ ]:
a=[i for i in [1,2,3] if i in [1,2]]


a

In [ ]:
next(flusso.df.iterrows())[1]['SERIE']